<a href="https://colab.research.google.com/github/NazariiDovhan19/sales-performance-analysis/blob/main/SalesKPI.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [4]:
import pandas as pd
import numpy as np
import os


# 1. Load dataset

file_path = "train.csv"  # твій файл
df = pd.read_csv(file_path)

print("Rows:", len(df))
print("Columns:", df.columns.tolist())


# 2. Cleaning

# Remove duplicates
df = df.drop_duplicates()

# Convert dates
df['Order Date'] = pd.to_datetime(df['Order Date'], dayfirst=True)
df['Ship Date'] = pd.to_datetime(df['Ship Date'], dayfirst=True)

# Remove invalid sales
df = df[df['Sales'] > 0]


# 3. Feature Engineering

df['order_year'] = df['Order Date'].dt.year
df['order_month'] = df['Order Date'].dt.to_period('M').astype(str)
df['shipping_days'] = (df['Ship Date'] - df['Order Date']).dt.days

# 4. KPI Summary

total_sales = df['Sales'].sum()
orders_count = df['Order ID'].nunique()
aov = total_sales / orders_count

kpi_summary = pd.DataFrame([{
    "total_sales": total_sales,
    "orders_count": orders_count,
    "average_order_value": aov,
    "average_shipping_days": df['shipping_days'].mean()
}])

print("\nKPI SUMMARY")
print(kpi_summary)

# 5. Monthly Sales Trend

monthly_sales = (
    df.groupby('order_month', as_index=False)
      .agg(
          total_sales=('Sales', 'sum'),
          orders=('Order ID', 'nunique')
      )
      .sort_values('order_month')
)

monthly_sales['aov'] = monthly_sales['total_sales'] / monthly_sales['orders']

# 6. Category Performance

category_sales = (
    df.groupby('Category', as_index=False)
      .agg(
          total_sales=('Sales', 'sum'),
          orders=('Order ID', 'nunique')
      )
      .sort_values('total_sales', ascending=False)
)

subcategory_sales = (
    df.groupby(['Category', 'Sub-Category'], as_index=False)
      .agg(
          total_sales=('Sales', 'sum'),
          orders=('Order ID', 'nunique')
      )
      .sort_values('total_sales', ascending=False)
)

# 7. Regional Performance

region_sales = (
    df.groupby('Region', as_index=False)
      .agg(
          total_sales=('Sales', 'sum'),
          orders=('Order ID', 'nunique'),
          avg_shipping_days=('shipping_days', 'mean')
      )
      .sort_values('total_sales', ascending=False)
)

state_sales = (
    df.groupby(['Region', 'State'], as_index=False)
      .agg(
          total_sales=('Sales', 'sum'),
          orders=('Order ID', 'nunique')
      )
      .sort_values('total_sales', ascending=False)
)

# 8. Customer Segment Analysis

segment_sales = (
    df.groupby('Segment', as_index=False)
      .agg(
          total_sales=('Sales', 'sum'),
          orders=('Order ID', 'nunique')
      )
      .sort_values('total_sales', ascending=False)
)

# 9. Export for Power BI

os.makedirs("outputs", exist_ok=True)

df.to_csv("outputs/orders_clean.csv", index=False)
kpi_summary.to_csv("outputs/kpi_summary.csv", index=False)
monthly_sales.to_csv("outputs/monthly_sales.csv", index=False)
category_sales.to_csv("outputs/category_sales.csv", index=False)
subcategory_sales.to_csv("outputs/subcategory_sales.csv", index=False)
region_sales.to_csv("outputs/region_sales.csv", index=False)
state_sales.to_csv("outputs/state_sales.csv", index=False)
segment_sales.to_csv("outputs/segment_sales.csv", index=False)

print("\nFiles exported to /outputs")


Rows: 9800
Columns: ['Row ID', 'Order ID', 'Order Date', 'Ship Date', 'Ship Mode', 'Customer ID', 'Customer Name', 'Segment', 'Country', 'City', 'State', 'Postal Code', 'Region', 'Product ID', 'Category', 'Sub-Category', 'Product Name', 'Sales']

KPI SUMMARY
    total_sales  orders_count  average_order_value  average_shipping_days
0  2.261537e+06          4922           459.475169               3.961122

Files exported to /outputs
